In [ ]:
import numpy as np
import random 
import math

In [ ]:
class Value:
    def __init__(self, data, _children=(), _op=""):
        self.data = data
        self.grad = 0 
        self._prev = set(_children)
        self._op = _op
        self._backward = lambda : None

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out
    
    # def __sub__(self, other):
    #     other = other if isinstance(other, Value) else Value(other)
    #     out = Value(self.data - other.data, (self, other), '-')
    #     return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.data += other.grad * out.grad
            other.data += self.data * out.grad
        out._backward = _backward

        return out
    
    def __pow__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data**other, (self, other), f"**{other}")

        def _backward():
            self.data += (other * self.data**(other-1)) * out.data
        out._backward = _backward
        
        return out
    
    def tanh(self):
        n = self.data
        t = (math.exp(2*n) - 1) / (math.exp(2*n) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():            # 1 + tanh ** 2
            self.data += (1 - t**2) * out.data
        out._backward = _backward

        return out


    def backward(self):
        #topological order
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1
        for v in reversed(topo):
            v._backward()


    def __repr__(self):
        return f"Value(data={self.data})"


In [36]:
class Neuron:
    def __init__(self, nin):
        self.w = []
        for _ in range(nin):
            val = random.uniform(-1, 1)
            self.w.append(Value(val))
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out


class Layers:
    def __init__(self, nin, nout):
        self.neurons = []           #[Neuron(nin) for _ in range(nout)]
        for _ in range(nout):
            self.neurons.append(Neuron(nin))

    def __call__(self, x):
        outs = []
        for n in self.neurons:
            outs.append(n(x))
        return outs
    

class MLP:
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = []
        for i in range(len(nouts)):
            nin = sz[i]
            nout = sz[i+1]
            #nonlin = i != len(nouts) - 1
            layer = Layers(nin, nout)  #layer = Layers(nin, nout, nonlin=nonlin)
            self.layers.append(layer)

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x



In [37]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 1])
n(x)

[Value(data=-0.09559296717940755)]